# **Coding Project: Encryption in Python**
---

### **Description**
In this project, you will be writing code to encrypt and decrypt a message of your choice using a key generated through QKD.

<br>

In [ ]:
!pip install cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hmac
from base64 import b64encode, b64decode
!pip install cirq --quiet
import cirq
from random import choices

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
def generate_bytes_from_bits(bits):
    # Join bits into a single string
    bit_string = ''.join(str(bit) for bit in bits)
    # Pad the bit string to make its length divisible by 8
    padded_bit_string = bit_string + '0' * (8 - len(bit_string) % 8)
    # Convert the bit string to bytes
    byte_data = bytes(int(padded_bit_string[i:i+8], 2) for i in range(0, len(padded_bit_string), 8))
    return byte_data

In [ ]:
def QKD(num_bits):
  #Setup
  encode_gates = {0: cirq.I, 1: cirq.X}
  basis_gates = {'Z': cirq.I, 'X': cirq.H}

  qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

  #Alice Chooses Bits and Bases
  alice_key = choices([0, 1], k = num_bits)
  alice_bases = choices(['Z', 'X'], k = num_bits)

  #Alice Creates Qubits
  alice_circuit = cirq.Circuit()

  for bit in range(num_bits):

    encode_value = alice_key[bit]
    encode_gate = encode_gates[encode_value]

    basis_value = alice_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    alice_circuit.append(encode_gate(qubit))
    alice_circuit.append(basis_gate(qubit))

  #Bob chooses a Bases
  bob_bases = choices(['Z', 'X'], k = num_bits)

  bob_circuit = cirq.Circuit()

  for bit in range(num_bits):

    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))

  #Bob Measures Qubits
  bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

  #Bob Creates a Key
  bb84_circuit = alice_circuit + bob_circuit

  sim = cirq.Simulator()
  results = sim.run(bb84_circuit)
  bob_key = results.measurements['bob key'][0]

  final_alice_key = []
  final_bob_key = []

  #Compare Bases
  for bit in range(num_bits):

    if alice_bases[bit] == bob_bases[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])

  #Compare Half their Bits
  num_bits_to_compare = int(len(final_alice_key) * .5)
  if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
    final_alice_key = final_alice_key[num_bits_to_compare:]
    final_bob_key = final_bob_key[num_bits_to_compare:]

    print('\n\nWe can use our keys!')
    print('Alice Key: ', final_alice_key)
    print('Bob Key: ', final_bob_key)

  else:
    print('\n\nEve was listening, we need to use a different channel!')
  # Assuming final_alice_key and final_bob_key are lists of bits
  alice_bytes = generate_bytes_from_bits(final_alice_key)
  bob_bytes = generate_bytes_from_bits(final_bob_key)

  return alice_bytes

### **Step 1**
You will first need to create some data we would like to send.

In [ ]:
unencrypted_string = "Meet me at the park tomorrow afternoon"

### **Step 2**
Use the `QKD` function, defined above, to create a key for your data.

In [ ]:
# Generate a random key (16 bytes for AES)
alice_bytes = QKD(512)
qkd_key = alice_bytes[:16]
print(qkd_key)

#len(qkd_key)



We can use our keys!
Alice Key:  [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
Bob Key:  [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
b'\xb9\xce^\xfc\x8ev\xc2\x16\xda\xdb\x8b\xdew\xd9\xe7\xf6'


### **Step 3**
---
You will need to create a function that can now ecrypt your message using your key. You may import from any python libraries you like to define this function.

In [ ]:
def encrypt_message(unencrypted_string, qkd_key):
  try:
    # Pad the plaintext message to match the block size of AES
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_message = padder.update(unencrypted_string.encode()) + padder.finalize()
    # Encrypt the padded message
    cipher = Cipher(algorithms.AES(qkd_key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()

    return ciphertext.hex()

  except (ValueError, AttributeError) as e:
    print("Error during Encryption:", e)
    return None

### **Step 4**

You will need to create a function that can now decrypt your message using your key. You may import from any python libraries you like to define this function.

In [ ]:
def decrypt_message(encrypted_message, qkd_key):
    try:
      # Convert encrypted_message from hexadecimal string to bytes
      ciphertext = bytes.fromhex(encrypted_message)

      # Decrypt the ciphertext
      cipher = Cipher(algorithms.AES(qkd_key), modes.ECB(), backend=default_backend())
      decryptor = cipher.decryptor()
      decrypted_message = decryptor.update(ciphertext) + decryptor.finalize()

      # Unpad the decrypted message to remove the padding
      unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
      original_plaintext = unpadder.update(decrypted_message) + unpadder.finalize()

      # Return the decrypted plaintext
      return original_plaintext.decode()

    except (ValueError, AttributeError) as e:
      print("Error during Decryption:", e)
      return None

### **Step 5**
---
Write code to encrypt and decrypt your message using your key to ensure that you were successful.

In [ ]:
# Encrypt the message
encrypted_message = encrypt_message(unencrypted_string, qkd_key)
print("Encrypted Message:", encrypted_message)

# Decrypt the message
decrypted_message = decrypt_message(encrypted_message, qkd_key)
print("Decrypted Message:", decrypted_message)

Encrypted Message: d69f341e4bc0954dbd6c5e3543eff2cb087eead07fa03c115b1d7803c4dee2dfe5b1e2fce6000046583f114dfe2108ba
Decrypted Message: Meet me at the park tomorrow afternoon


# End of Notebook

---
© 2024 The Coding School, All rights reserved